In [ ]:
import pandas as pd
import polars as pl
from datetime import datetime, timedelta, date
semana = date.today().isocalendar().week

In [81]:
clubs = (pl.read_csv("https://raw.githubusercontent.com/IvoVillanueva/acb2026/refs/heads/main/logos_calendario/clubs2026.csv")
    .with_columns(
        pl.col("equipo").str.replace("Baskonia", "Baskonia Kosner"))
    .with_columns([
        pl.when(pl. col("equipo") == "Real Madrid")
          .then(pl.lit("#00529F"))
          .otherwise(pl.col("color"))
          .alias("color2"),
        
        pl.when(pl.col("equipo") == "Baskonia Kosner")
          .then(pl.lit("#000000"))
          .otherwise(pl.col("color"))
          .alias("color"),
        
        pl.when(pl.col("equipo") == "Baskonia Kosner")
          .then(pl.lit("https://static.acb.com/img/www/clubes2025/2526KosnerBaskoniaLogoNegativo.png"))
          .otherwise(pl.col("logo"))
          .alias("logo")
    ]))
clubs

equipo,logo,color,url,abb,color2
str,str,str,str,str,str
"""Barça""","""https://static.acb.com/img/www…","""#154284""","""https://www.acb.com/club/plant…","""BAR""","""#154284"""
"""Baskonia Kosner""","""https://static.acb.com/img/www…","""#000000""","""https://www.acb.com/club/plant…","""BKN""","""#C8102E"""
"""Bàsquet Girona""","""https://static.acb.com/img/www…","""#B32836""","""https://www.acb.com/club/plant…","""GIR""","""#B32836"""
"""BAXI Manresa""","""https://static.acb.com/img/31/…","""#D5001E""","""https://www.acb.com/club/plant…","""BAX""","""#D5001E"""
"""Casademont Zaragoza""","""https://static.acb.com/img/www…","""#C8102E""","""https://www.acb.com/club/plant…","""CAZ""","""#C8102E"""
…,…,…,…,…,…
"""Río Breogán""","""https://static.acb.com/logos/1…","""#85ADEA""","""https://www.acb.com/club/plant…","""BRE""","""#85ADEA"""
"""Surne Bilbao Basket""","""https://static.acb.com/img/www…","""#000000""","""https://www.acb.com/club/plant…","""SBB""","""#000000"""
"""UCAM Murcia""","""https://static.acb.com/img/www…","""#A51A14""","""https://www.acb.com/club/plant…","""UCM""","""#A51A14"""


In [82]:
data_euroleague = (
    pl.read_csv(
        "https://raw.githubusercontent.com/IvoVillanueva/BOXSCORES-EUROLEAGE-2025_26/refs/heads/main/data/euroleague_boxscore_2025_26.csv"
    )
    .with_columns(
        pl.col("fecha")
          .str.to_datetime("%Y-%m-%d %H:%M:%S")
          .alias("fecha"),

        pl.col("minutes")
          .str.replace("DNP", "0:00")
          .alias("minutes")
    )
    .with_columns(
        # --- Normalizar nombres de equipo ---
        pl.when(pl.col("team_name") == "KOSNER BASKONIA VITORIA-GASTEIZ")
          .then(pl.lit("Baskonia Kosner"))
          .when(pl.col("team_name") == "REAL MADRID")
          .then(pl.lit("Real Madrid"))
          .when(pl.col("team_name") == "FC BARCELONA")
          .then(pl.lit("Barça"))
          .when(pl.col("team_name") == "VALENCIA BASKET")
          .then(pl.lit("Valencia Basket"))
          .otherwise(pl.col("team_name"))
          .alias("team_name"),

        # --- firstName ---
        pl.when(pl.col("player").str.contains(","))
          .then(
              pl.col("player")
                .str.split(", ")
                .list.get(1)
                .str.to_titlecase()
          )
          .otherwise(pl.col("player").str.to_titlecase())
          .alias("firstName"),

        # --- lastName ---
        pl.when(pl.col("player").str.contains(","))
          .then(
              pl.col("player")
                .str.split(", ")
                .list.get(0)
                .str.to_titlecase()
          )
          .otherwise(pl.lit(None))
          .alias("lastName"),

        # --- nombre completo ---
        pl.when(pl.col("player").str.contains(","))
          .then(
              pl.col("player").str.split(", ").list.get(1).str.to_titlecase()
              + pl.lit(" ")
              + pl.col("player").str.split(", ").list.get(0).str.to_titlecase()
          )
          .otherwise(pl.col("player").str.to_titlecase())
          .alias("player"),

        # --- minutos redondeados ---
        (
            pl.col("minutes")
              .str.split(":")
              .list.get(0)
              .cast(pl.Int64)
            +
            pl.when(
                pl.col("minutes")
                  .str.split(":")
                  .list.get(1)
                  .cast(pl.Int64) >= 30
            )
            .then(1)
            .otherwise(0)
        ).alias("minutes")
    )
    .filter(
        (pl.col("team_name").is_in([
            "Baskonia Kosner",
            "Real Madrid",
            "Barça",
            "Valencia Basket"
        ]))
        & (pl.col("ronda") == pl.col("ronda").max())
        & (pl.col("semana") == semana)   
    )
    .select([
        "competicion",
        "fecha",
        "semana",
        "ronda",
        pl.col("id_match").alias("partido"),
        "firstName",
        "lastName",
        "player",
        pl.col("team_name").alias("equipo"),
        pl.col("minutes").alias("min"),
        pl.col("points").alias("pts"),
        pl.col("total_rebounds").alias("reb"),
        pl.col("assistances").alias("ast"),
        pl.col("steals").alias("br"),
        pl.col("turnovers").alias("bp"),
        pl.col("blocks_favour").alias("tap"),
        pl.col("fouls_commited").alias("fa"),
        pl.col("valuation").alias("val"),
        pl.col("plusminus").alias("mm")
    ])
)

data_euroleague


competicion,fecha,semana,ronda,partido,firstName,lastName,player,equipo,min,pts,reb,ast,br,bp,tap,fa,val,mm
str,datetime[μs],i64,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""Euroleague""",2025-12-30 19:30:00,1,19,181,"""Kevin""","""Punter""","""Kevin Punter""","""Barça""",30,8,2,4,2,3,0,1,6,-7
"""Euroleague""",2025-12-30 19:30:00,1,19,181,"""Juani""","""Marcos""","""Juani Marcos""","""Barça""",6,0,0,2,0,0,0,1,-2,-4
"""Euroleague""",2025-12-30 19:30:00,1,19,181,"""Myles""","""Cale""","""Myles Cale""","""Barça""",18,7,3,1,1,1,0,2,9,-6
"""Euroleague""",2025-12-30 19:30:00,1,19,181,"""Miles""","""Norris""","""Miles Norris""","""Barça""",29,14,4,2,0,0,0,2,14,-4
"""Euroleague""",2025-12-30 19:30:00,1,19,181,"""Jan""","""Vesely""","""Jan Vesely""","""Barça""",6,0,0,0,0,2,0,2,-3,-13
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Euroleague""",2026-01-02 19:45:00,1,19,189,"""Usman""","""Garuba""","""Usman Garuba""","""Real Madrid""",15,4,3,1,0,2,2,3,6,7
"""Euroleague""",2026-01-02 19:45:00,1,19,189,"""Walter""","""Tavares""","""Walter Tavares""","""Real Madrid""",25,6,6,1,1,1,4,3,16,7
"""Euroleague""",2026-01-02 19:45:00,1,19,189,"""Sergio""","""Llull""","""Sergio Llull""","""Real Madrid""",7,0,1,4,0,1,0,0,5,-8


In [80]:
df_partidos_euroleague = (
    data_euroleague
    .group_by(["player", "equipo"])
    .agg([
        pl.col("min").mean(),
        pl.col("pts").mean(),
        pl.col("reb").mean(),
        pl.col("ast").mean(),
        pl.col("br").mean(),
        pl.col("bp").mean(),
        pl.col("tap").mean(),
        pl.col("fa").mean(),
        pl.col("mm").mean(),
        pl.col("val").mean()
    ])
    .sort("player")
    .join(clubs, on="equipo")
)
df_partidos_euroleague

player,equipo,min,pts,reb,ast,br,bp,tap,fa,mm,val,logo,color,url,abb,color2
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str
"""Alberto Abalde""","""Real Madrid""",19.0,14.0,3.0,0.0,2.0,1.0,0.0,0.0,15.0,18.0,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB""","""#00529F"""
"""Alex Len""","""Real Madrid""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB""","""#00529F"""
"""Andres Feliz""","""Real Madrid""",10.0,5.0,2.0,0.0,0.0,1.0,0.0,4.0,-4.0,-2.0,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB""","""#00529F"""
"""Brancou Badio""","""Valencia Basket""",24.0,14.0,1.0,3.0,1.0,3.0,0.0,2.0,11.0,9.0,"""https://static.acb.com/img/23/…","""#FC6C0F""","""https://www.acb.com/club/plant…","""VBC""","""#FC6C0F"""
"""Braxton Key""","""Valencia Basket""",5.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,-1.0,-1.0,"""https://static.acb.com/img/23/…","""#FC6C0F""","""https://www.acb.com/club/plant…","""VBC""","""#FC6C0F"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Trey Lyles""","""Real Madrid""",19.0,17.0,4.0,1.0,0.0,1.0,0.0,0.0,8.0,21.0,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB""","""#00529F"""
"""Usman Garuba""","""Real Madrid""",15.0,4.0,3.0,1.0,0.0,2.0,2.0,3.0,7.0,6.0,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB""","""#00529F"""
"""Walter Tavares""","""Real Madrid""",25.0,6.0,6.0,1.0,1.0,1.0,4.0,3.0,7.0,16.0,"""https://static.acb.com/img/www…","""#FFFFFF""","""https://www.acb.com/club/plant…","""RMB""","""#00529F"""
